In [1]:
cd lag-llama

/home/user/energygpt/lagllama/lag-llama


/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import glob
from collections import defaultdict
from datetime import datetime
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

import warnings  
warnings.filterwarnings('ignore') 


In [3]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=168, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("./checkpoints/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="./checkpoints/lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=32,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [ ]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples
  

In [ ]:
def forecast_building(df):

    # Set numerical columns as float32
    for col in df.columns:
        # Check if column is not of string type
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')
    
    # Create the Pandas
    dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")
    
    backtest_dataset = dataset
    prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
    num_samples = 10 # number of samples sampled from the probability distribution for each timestep
    device = torch.device("cuda:0") # You can switch this to CPU or other GPUs if you'd like, depending on your environment    
    
    ckpt = torch.load("./checkpoints/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.

    forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples=num_samples)

    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))     
    
    res_all = []
    for ts, fc in zip(tss, forecasts):
        res = ts[ts.index.isin(fc.index)]
        res.columns = ['y_true']
        res.insert(1, 'y_pred', fc.median)
        res_all.append(res)
    res_all_df = pd.concat(res_all).sort_index()
    return res_all_df, agg_metrics, ts_metrics 

    
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    res, agg_metrics, ts_metrics = forecast_building(windows_all_df)
    return res, agg_metrics, ts_metrics


# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    res_all = []
    agg_metrics_all = []
    ts_metrics_all = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]#.head(24*200)

        res, agg_metrics, ts_metrics = process_building(df1)
        res['building'] = building_name
        res['filename'] = filename
        res_all.append(res)
        
        ts_metrics.insert(0, 'building', building_name)
        ts_metrics.insert(0, 'filename', filename)
        ts_metrics = ts_metrics.sort_values(['forecast_start'])
        ts_metrics_all.append(ts_metrics)
        
        agg_metrics_df = pd.DataFrame([agg_metrics])
        agg_metrics_df.insert(0, 'building', building_name)
        agg_metrics_df.insert(0, 'filename', filename)
        agg_metrics_all.append(agg_metrics_df)

        i += 1
        if i % 10 == 0:
            print(datetime.now(), 'Saving...')
            res_all_df = pd.concat(res_all).round(6)
            res_all_df = res_all_df.reset_index()
            res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
            res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

            ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)
            ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

            agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)            
            agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)            
    
    
    res_all_df = pd.concat(res_all).round(6)
    res_all_df = res_all_df.reset_index()
    res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
    res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

    ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)    
    ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

    agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)   
    agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)                

    return res_all_df, ts_metrics_all_df, agg_metrics_all_df

In [18]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/*csv')

dataset = 'Bareilly'
os.makedirs(f'../forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'../results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    # if results is not None:
    #     results.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)
    print('')

2024-10-26 10:36:01.157757 /home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/Bareilly_2021.csv
2024-10-26 10:36:01.186321 (7296, 38)
2024-10-26 10:36:01.186810 0 / 38 BR02


Running evaluation: 296it [00:00, 5943.27it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:154: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:4

2024-10-26 10:36:34.371184 1 / 38 BR04


Running evaluation: 296it [00:00, 6110.84it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:154: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:4

2024-10-26 10:37:08.109248 2 / 38 BR05


...

Running evaluation: 358it [00:00, 8008.15it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluati

2024-10-26 11:24:27.659592 44 / 46 BR51


Running evaluation: 358it [00:00, 7493.01it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluati

2024-10-26 11:25:04.674802 45 / 46 BR52


Running evaluation: 358it [00:00, 8027.03it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in float_scalars
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/evaluati

## Metrics

In [19]:
#!pip install permetrics

In [20]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Bareilly'
files_list = glob.glob(f'../forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        #data = [data.y_pred >= 0]
        print(g)  
        if data.empty:
            continue 
            
        rmse= root_mean_squared_error(data.y_true, data.y_pred)
        mae= mean_absolute_error(data.y_true, data.y_pred)
        mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
        mse= mean_squared_error(data.y_true, data.y_pred)
        msle= mean_squared_log_error(data.y_true, data.y_pred)
        rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)

        evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
        nrmse = evaluator.normalized_root_mean_square_error()
    
        metrics = pd.DataFrame({'building_name': [g[0]], 
                           'mae': [mae],
                            'mape': [mape],
                           'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse]})
        metrics_all.append(metrics)
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'../results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

('BR02',)
('BR04',)
('BR05',)
('BR06',)
('BR08',)
('BR09',)
('BR11',)
('BR12',)
('BR13',)
('BR15',)
('BR16',)
('BR18',)
('BR19',)
('BR22',)
('BR24',)
('BR27',)
('BR28',)
('BR29',)
('BR30',)
('BR31',)
('BR32',)
('BR33',)
('BR34',)
('BR35',)
('BR36',)
('BR37',)
('BR38',)
('BR39',)
('BR42',)
('BR43',)
('BR44',)
('BR45',)
('BR46',)
('BR48',)
('BR49',)
('BR50',)
('BR51',)
('BR52',)
('BR02',)
('BR03',)
('BR04',)
('BR05',)
('BR06',)
('BR07',)
('BR08',)
('BR09',)
('BR10',)
('BR11',)
('BR12',)
('BR13',)
('BR14',)
('BR15',)
('BR16',)
('BR17',)
('BR18',)
('BR19',)
('BR20',)
('BR22',)
('BR23',)
('BR24',)
('BR26',)
('BR27',)
('BR28',)
('BR29',)
('BR30',)
('BR31',)
('BR32',)
('BR33',)
('BR34',)
('BR35',)
('BR36',)
('BR37',)
('BR38',)
('BR39',)
('BR42',)
('BR43',)
('BR44',)
('BR45',)
('BR46',)
('BR48',)
('BR49',)
('BR50',)
('BR51',)
('BR52',)


In [21]:
metrics_all_files_df.to_csv(f'../results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,filename
0,BR02,0.007392,1.869375e+12,0.001556,0.039444,0.001108,0.033284,2.182239,Bareilly_2021.csv
0,BR04,0.186246,4.909827e+13,0.082376,0.287013,0.037304,0.193143,1.403834,Bareilly_2021.csv
0,BR05,0.024703,1.512328e+13,0.003136,0.055996,0.002445,0.049446,1.021208,Bareilly_2021.csv
0,BR06,0.423357,4.630188e+13,0.364166,0.603462,0.107709,0.328190,2.010850,Bareilly_2021.csv
0,BR08,0.009553,3.599372e+12,0.002188,0.046772,0.001469,0.038333,2.415613,Bareilly_2021.csv
...,...,...,...,...,...,...,...,...,...
0,BR48,0.039865,1.412722e+13,0.004607,0.067876,0.003553,0.059608,0.933128,Bareilly_2020.csv
0,BR49,0.244413,4.551264e+13,0.159663,0.399579,0.048702,0.220685,0.949559,Bareilly_2020.csv
0,BR50,0.123642,5.681389e+13,0.039163,0.197896,0.023323,0.152720,1.605140,Bareilly_2020.csv
0,BR51,0.420393,1.040113e+14,0.384986,0.620472,0.093097,0.305118,1.134996,Bareilly_2020.csv
